# Preprocessing

We use `pandas` to help with preprocessing.
The goal is to have the variables in TABLE 1 of the paper and no more.
The steps we take:
1) Read in the dataset, only the columns that we will need.
1) Drop rows where `Sales` is either zero or missing.
    Since `Sales` is our output variable, it makes sense to drop rows missing it.
1) Make a transformation to the `Date` column to mimic the paper.
1) Merge in the `State` column from a separate sheet.
    Initially it's in a textual form, encode it into numeric values.
1) Make all the categorical variables start from 0.
    It's a requirement to create embeddings correctly, and although this can be done at a later stage, we find it convenient to do it here.
1) Rename and reorder the columns, for no reason really.

In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

dataset = pd.read_csv("rossmann-store-sales/train.csv", usecols=["Store", "DayOfWeek", "Date", "Sales", "Promo"])

# Drop rows for which we don't have sales figures
dataset = dataset[(dataset["Sales"] != 0) & (dataset["Sales"].notna())]

# Split Date into Day, Month, Year
dataset[["Year", "Month", "Day"]] = dataset["Date"].str.split("-", expand=True).astype(int)
dataset.drop(columns=["Date"], inplace=True)

# Add state column
state_df = pd.read_csv("rossmann-store-sales/store_states.csv")
dataset = pd.merge(dataset, state_df, how="left", on="Store")
del state_df

# Encode state as integers
label_encoder = LabelEncoder()
dataset["State"] = label_encoder.fit_transform(dataset["State"])

# Shift the range of all categorical variables to start from 0.
# This is needed to create embeddings and convenient to do here
dataset[dataset.columns.difference(['Sales'])] -= dataset[dataset.columns.difference(['Sales'])].min()

# Rename and reorder columns
dataset.columns = dataset.columns.str.lower()
dataset.rename(columns={"dayofweek": "day_of_week", "promo": "promotion"}, inplace=True)
dataset = dataset[["store", "day_of_week", "day", "month", "year", "promotion", "state", "sales"]]

# Creating Tensors

Mostly straighforward. We highlight the notable bits.

Firsly, the paper describes a transformation to make to the output variable so it is within the range of the sigmoid function.
We create a `OutputEncoder` class to encapsulate this so it is easy to do this transformation in both directions.

Next we do the test/train split. The paper describes two ways to do it:
1) Preseving original temporal ordering
2) Shuffling the data

We implement both and state both results

In [9]:
import torch

X = torch.tensor(dataset.drop(columns=["sales"]).values, dtype=torch.int64)
y = torch.tensor(dataset["sales"].values, dtype=torch.float)

In [10]:
class OutputEncoder():
    def __init__(self, max_output):
        self.max_output = max_output

    def encode(self, output):
        with torch.no_grad():
            return torch.log(output) / torch.log(self.max_output)

    def decode(self, output):
        with torch.no_grad():
            return torch.exp(output * torch.log(self.max_output))

output_encoder = OutputEncoder(torch.max(y))
y = output_encoder.encode(y)

In [11]:
# Common split function
def test_train_split(X, y):
    split_threshold = int(0.9 * X.size(0))
    
    X_train = X[:split_threshold]
    X_test = X[split_threshold:]

    y_train = y[:split_threshold]
    y_test = y[split_threshold:]

    return X_train, y_train, X_test, y_test

# Temporal split (Already in correct order)
X_temporal = X.clone()
y_temporal = y.clone()

# Shuffled split
shuffled_indices = torch.randperm(X.size(0))

X_shuffled = X[shuffled_indices].clone()
y_shuffled = y[shuffled_indices].clone()

# Creating Neural Networks

The architecture is defined well in the paper and we will follow it. The paper defines 2 ways to create the networks, with embeddings and with one-hot vectors. We again implement both ways.

In [12]:
class EmbeddingNN(torch.nn.Module):
    def __init__(self):
        super(EmbeddingNN, self).__init__()
        # From TABLE 1. Each tuple is (unique_values, embedding_dimension)
        ee_dims = [(1115, 10), (7, 6), (31, 10), (12, 6), (3, 2), (2, 1), (12, 6)]
        total_emb_dim = sum(dim for _, dim in ee_dims)

        self.ee_layers = [torch.nn.Embedding(*args) for args in ee_dims]
        self.fc1 = torch.nn.Linear(total_emb_dim, 1000)
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(1000, 500)
        self.relu2 = torch.nn.ReLU()
        self.output = torch.nn.Linear(500, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, X):
        embedded = [ee_layer(X[:, i]) for i, ee_layer in enumerate(self.ee_layers)]
        embedded = torch.cat(embedded, dim=1)

        out = self.relu1(self.fc1(embedded))
        out = self.relu2(self.fc2(out))
        out = self.sigmoid(self.output(out))
        return out

class OneHotNN(torch.nn.Module):
    def __init__(self):
        super(OneHotNN, self).__init__()
        # Required to create the one-hot vectors
        self.one_hot_classes = [1115, 7, 31, 12, 3, 2, 12]
        self.total_classes = sum(self.one_hot_classes)

        self.fc1 = torch.nn.Linear(self.total_classes, 1000)
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(1000, 500)
        self.relu2 = torch.nn.ReLU()
        self.output = torch.nn.Linear(500, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, X):
        one_hot = [torch.nn.functional.one_hot(X[:, i], num_class).float() for i, num_class in enumerate(self.one_hot_classes)]
        one_hot = torch.cat(one_hot, dim=1)

        out = self.relu1(self.fc1(one_hot))
        out = self.relu2(self.fc2(out))
        out = self.sigmoid(self.output(out))
        return out

# Training and Testing

We create some functions to reduce repetitive code when creating multiple models and training on different data.
Important things to note are that for predictions, 5 models are created and trained, then their predictions averaged, as mentioned in the paper.
The `MAPE` (mean absolute percent error) metric is used for scoring.

In [13]:
def train_model(model, X, y):
    loss_fn = torch.nn.MSELoss()
    optim = torch.optim.Adam(model.parameters(), lr=0.001)

    epochs = 10
    batch_size = 1024
    total_samples = len(X)

    model.train()
    for _ in range(epochs):
        for i in range(0, total_samples, batch_size):
            inputs = X[i:i+batch_size]
            targets = y[i:i+batch_size]
            
            optim.zero_grad()

            outputs = model(inputs).squeeze()
            loss = loss_fn(outputs, targets)
        
            loss.backward()

            optim.step()

def MAPE(models, X, y_true):
    y_preds = [model(X).squeeze() for model in models]
    stacked_preds = torch.stack(y_preds)
    y_pred = torch.mean(stacked_preds, dim=0)

    y_pred = output_encoder.decode(y_pred)
    y_true = output_encoder.decode(y_true)

    return torch.mean(torch.abs((y_true - y_pred) / y_true))

def evaluate(cls, X, y):
    X_train, y_train, X_test, y_test = test_train_split(X, y)
    models = [cls() for _ in range(5)]

    for model in models:
        train_model(model, X_train, y_train)

    return MAPE(models, X_test, y_test)

In [14]:
print(f"Shuffled OneHotNN: {evaluate(OneHotNN, X_shuffled, y_shuffled):.3f}")
print(f"Shuffled EmbeddingNN: {evaluate(EmbeddingNN, X_shuffled, y_shuffled):.3f}")
print(f"Temporal OneHotNN: {evaluate(OneHotNN, X_temporal, y_temporal):.3f}")
print(f"Temporal EmbeddingNN: {evaluate(EmbeddingNN, X_temporal, y_temporal):.3f}")

Shuffled OneHotNN: 0.075
Shuffled EmbeddingNN: 0.082
Temporal OneHotNN: 0.122
Temporal EmbeddingNN: 0.185


# Results

We get the following results. Note that the numbers are `MAPE` score.

|  | OneHotNN | EmbeddingNN |
| --- | --- | --- |
| Shuffled Data | 0.075 | 0.082 |
| Temporal Data | 0.122 | 0.185 |

Compare with the paper's results

|  | OneHotNN | EmbeddingNN |
| --- | --- | --- |
| Shuffled Data | 0.070 | 0.070 |
| Temporal Data | 0.101 | 0.093 |

Our results are worse than theirs. At worst, twice as bad. It's not immediately clear why, we tried to implement their methodology faithfully.